In [ ]:
from preprocess import *

# Data import

In [ ]:
# import roads data
"""
The column named 'travelTimeIncreaseRatio' in the roads_shape.shp is truncated into 'travelTime' due to the length limited set by geopands.
This column is actually the travel distance increase ratio (equal to travel time increase ratio if vehicle speed is assumed to be same) we want to get.
So, this column is dropped at the beginning. It will be added again as travelTimeIncreaseRatio in the ending of computation.
"""
roads = readRoads('./data/roads/roads_shape.shp').drop(['travelTime'], axis = 1) 
roads_line = readRoads('./data/roads/roads_line.shp')
roads['line'] = roads_line['geometry']
roads = roads.rename(columns = {"geometry": "surface"})
roads = roads.set_geometry('surface')


# create graph from roads
graph = roads2Graph(roads) # NOTE: the graph is un-directed right now, the logic should be checked if changed to directed

# read the location of rescue squads and attach them to graph nodes 
rescue = readRescue('./data/rescueTeamLocation/rescueStations.txt', 'EPSG:4326', roads)

# Disruption analysis

In [ ]:
def _addPathLen2Graph(graph, rescue, weight, newAttribute_rescueSquad, newAttribute_path):
    # some roads are disconnected from all the rescue station even in normal time (as the raw data indicates)
    voronoi = nx.voronoi_cells(graph, set(rescue.OBJECTID_nearestRoad.unique()), weight = weight)
    for rescueSquad, destinations in zip(voronoi.keys(), voronoi.values()):
        if rescueSquad == 'unreachable':
            print(len(destinations), 'nodes are unreachable when building voronoi for', newAttribute_path)
            for des in destinations:
                graph.nodes[des][newAttribute_rescueSquad] = np.nan
                graph.nodes[des][newAttribute_path] = math.inf # set path len to inf if it's disconnected from rescues
#                 print('NOTE: node', des, 'is unreachable when building voronoi for', newAttribute_path)
        else:
            for des in destinations:
                shortestPath = nx.shortest_path_length(graph, source = rescueSquad, target = des, weight = weight)
                graph.nodes[des][newAttribute_path] = shortestPath
                graph.nodes[des][newAttribute_rescueSquad] = rescueSquad
                if shortestPath == 0:
                    graph.nodes[des][newAttribute_path] = 1
                if shortestPath == math.inf:
                    graph.nodes[des][newAttribute_rescueSquad] = math.inf
    return graph, voronoi

def _addDisruption(graph, roads, newAttribute = 'weightWithDisruption', threshold = 3):
    nx.set_edge_attributes(graph, nx.get_edge_attributes(graph, "weight"), newAttribute)
    disruptedRoads = roads[roads['waterDepth'] >= threshold]['OBJECTID'].to_list()
    for disruption in disruptedRoads:
        for edge in graph.edges(disruption):
            graph.edges()[edge][newAttribute] = math.inf # set edge weight to inf if it's disrupted by inundation
    return graph

def _changeValue4DisruptedRoad(roads, graph, threshold = 3):
    # the disrupted road itself is not disconnected, so assign the shortestPath of adjancent road to this road
    for disruption in roads[roads['waterDepth'] >= threshold]['OBJECTID'].to_list():
        pathLen = []
        edgeNum = []
        for edge in graph.edges(disruption):
            pathLen.append(graph.nodes()[edge[1]]['shortestPathLenWithDisruption'])
            edgeNum.append(edge[1])
        if pathLen != []: # in case there are disconnected single node
            graph.nodes()[disruption]['shortestPathLenWithDisruption'] = min(pathLen)
            if min(pathLen) != math.inf:
                graph.nodes()[disruption]['rescueAssignedWithDisruption'] = edgeNum[pathLen.index(min(pathLen))]
            else:
                graph.nodes()[disruption]['rescueAssignedWithDisruption'] = np.nan
    return graph

def runRoutingWithDisruption(graph, rescue, roads):
    graph, _ = _addPathLen2Graph(graph, rescue, 'weight', 'rescueAssigned', 'shortestPathLen')
    graphDisrupted = _addDisruption(graph, roads, threshold = 1)
    graph, _ = _addPathLen2Graph(graphDisrupted, rescue, 'weightWithDisruption', 'rescueAssignedWithDisruption', 'shortestPathLenWithDisruption') 
    graph = _changeValue4DisruptedRoad(roads, graph, threshold = 1)
    return graph

def getDisruptionRatio(graph):
    nx.set_node_attributes(graph, 
                           {x[0]: y[1]/x[1] if y[1]/x[1] != math.inf else np.nan \
                            for x, y in zip(nx.get_node_attributes(graph, "shortestPathLen").items(), 
                                            nx.get_node_attributes(graph, "shortestPathLenWithDisruption").items() ) },
                           'travelTimeIncreaseRatio')
    roads['travelTimeIncreaseRatio'] = roads['OBJECTID'].map(nx.get_node_attributes(graph, "travelTimeIncreaseRatio"))    
    return graph


In [ ]:
# calculate ratios
graph = runRoutingWithDisruption(graph, rescue, roads)
graph = getDisruptionRatio(graph)

# Visualization

In [ ]:
def showWaterOnRoads(roads, figsize = (100, 50), vmax = 6):
    fig, ax = plt.subplots(figsize = figsize)
    roadsLineWater = roads.loc[:, ['line', 'waterDepth']].set_geometry('line')
    ax = roadsLineWater.plot(ax = ax, 
                        column = 'waterDepth', 
                        zorder = 5, 
                        cmap = 'OrRd',
                        legend = True,
                        vmax = vmax,
                       )
    cx.add_basemap(ax, crs = roads.crs, source = cx.providers.CartoDB.Positron)
    ax.set_axis_off()
    
def showTravelUpRatioOnRoads(roads, figsize = (100, 50), vmax = 10):
    fig, ax = plt.subplots(figsize = figsize)
    roadsLineWater = roads.loc[:, ['line', 'travelTimeIncreaseRatio']].set_geometry('line')
    ax = roadsLineWater.plot(ax = ax, 
                        column = 'travelTimeIncreaseRatio', 
                        zorder = 5, 
                        cmap = 'OrRd',
                        legend = True,
                        vmax = vmax,
                        vmin = 1,
                       )
    cx.add_basemap(ax, crs = roads.crs, source = cx.providers.CartoDB.Positron)
    ax.set_axis_off()

In [ ]:
"""
The 2nd argument sets the size of the figure, which also determines th resolution of the figure.
The 3rd arguement set the maximum value that can be shown in the figure. There are some outliers in the results. 
    The visualization would be less informative if this arguement is missing or high.
"""
showWaterOnRoads(roads, (50, 25), 6) # the depth of water on roads
showTravelUpRatioOnRoads(roads, (20, 12), 10) # the travel distance/time increase ratio of roads